In [9]:
import pandas as pd
import json
import random

Abbiamo inizialmente convertito il file RAW_recipes.csv in RAW_recipes.json

Abbiamo pulito il file RAW_recipes.json, il quale ha la seguente struttura: 
-   name
-   id: id della ricetta  ----- (da cancellare dopo collegamenti)
-   minutes XX
-   contributor_id: utente
-   submitted: data (TIMESTAMP POST)
-   tags: magari si potrebbero droppare, non servono XX
-   nutrition: valori nutrizionali associati alla ricetta, magari si può droppare anche questa, si può ricavare facilmente dalle informazioni in ingredienti XX
-   n_steps XX
-   steps
-   description (DESCRIPTION POST)
-   ingredients
-   n_ingredients XX

in:
-   name
-   id: id della ricetta  ----- (da cancellare dopo collegamenti)
-   contributor_id: utente
-   submitted: data (TIMESTAMP POST)
-   steps
-   description (DESCRIPTION POST)
-   ingredients

In [8]:
# Leggi il JSON in un DataFrame
raw_recipes_df = pd.read_json('RAW_recipes.json', orient='records')

# Elimina le colonne non necessarie
columns_to_drop = ['minutes', 'tags', 'nutrition', 'n_steps', 'n_ingredients']
reduced_df = raw_recipes_df.drop(columns=columns_to_drop)

# Definizione del percorso del nuovo file JSON
new_json_path = 'RAW_recipes_reduced.json'

# Salvataggio del DataFrame in un nuovo file JSON con formattazione pretty
reduced_df.to_json(new_json_path, orient='records', lines=False, indent=4)


Abbiamo trasformato steps e ingredients da stringa in array di stringhe

In [9]:
# Funzione per convertire stringhe in array di stringhe (separatore: "', '")
def convert_str_to_array(string):
    return string[2:-2].split("', '")

reduced_df['steps'] = reduced_df['steps'].apply(convert_str_to_array)
reduced_df['ingredients'] = reduced_df['ingredients'].apply(convert_str_to_array)

# Definizione del percorso del nuovo file JSON
new_json_path = 'RAW_recipes_reduced_array.json'

# Salvataggio del DataFrame in un nuovo file JSON con formattazione pretty
reduced_df.to_json(new_json_path, orient='records', lines=False, indent=4)

Passiamo alle modifiche del file recipes.csv, da cui recupereremo le immagini:
-   RecipeId
-   AuthorName
-   Image URLs

In [12]:
image_recipes_df = pd.read_json('IMAGE_recipes.json', orient='records')

column_to_keep = ['RecipeId', 'AuthorName', 'Images']

reduced_image_df = image_recipes_df[column_to_keep]

new_json_path = 'IMAGE_recipes_reduced.json'

reduced_image_df.to_json(new_json_path, orient='records', lines=False, indent=4)

In [21]:
def extract_first_image(images_field):
    if images_field == 'character(0)' or images_field == None:
        return ""
    images = images_field.split('", ')
    if(len(images) == 0):
        return images_field
    first_image = images[0].split('(')[-1].strip('\"')
    return first_image

# Applica la funzione alla colonna 'Images'
reduced_image_df['Image'] = reduced_image_df['Images'].apply(extract_first_image)

# Seleziona solo le colonne necessarie
df_output = reduced_image_df[['RecipeId', 'AuthorName', 'Image']]

new_json_path = 'IMAGE_recipes_reduced_single_image.json'

df_output.to_json(new_json_path, orient='records', lines=False, indent=4)

/var/folders/rv/w7k68zds33v40x4d_lx8k5rh0000gn/T/ipykernel_31432/745219229.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_image_df['Image'] = reduced_image_df['Images'].apply(extract_first_image)


ESEMPIO:
https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/YUeirxMLQaeE1h3v3qnM_229%20berry%20blue%20frzn%20dess.jpg

Se la stringa di Image è vuota, verrà messa l'immagine di default (logo di WeFood)

A questo punto, si collegano le immagini al file iniziale delle ricette RAW_recipes_reduced_array.json

In [3]:
with open('RAW_recipes_reduced_array.json', 'r') as file:
    dati1 = json.load(file)

# Carica il secondo file JSON
with open('IMAGE_recipes_reduced_single_image.json', 'r') as file:
    dati2 = json.load(file)

# Converti il secondo set di dati in un dizionario per un accesso più rapido
dati2_dict = {item['RecipeId']: item for item in dati2}

# Esegui il merge
dati_combinati = []
for item in dati1:
    id_ricetta = item['id']
    if id_ricetta in dati2_dict:
        # Combina i dati
        dati_combinati.append({**item, **dati2_dict[id_ricetta]})

# Salva il risultato in un nuovo file JSON
with open('RAW_recipes_images.json', 'w') as file:
    json.dump(dati_combinati, file, indent=4)

In [8]:
len(dati_combinati)

231323

Il file ha 231323 ricette

Si procede con salvare gli ingredienti con la struttura che avranno nel post all'interno del file RAW_recipes_images.json

In [14]:
with open('RAW_recipes_images.json', 'r') as file:
    ricette = json.load(file)

# Carica il secondo file JSON
with open('../ingredients/merged_fuzzy_df_updated_reduced.json', 'r') as file:
    ingredienti_dati = json.load(file)

for ricetta in ricette:
    nuovi_ingredienti = []
    for ingrediente in ricetta['ingredients']:
        # Qui assumeremo che se una parola chiave in 'ingrediente' è presente in 'raw_ingr', c'è un match
        # Questo è un esempio molto semplice e potrebbe non funzionare in tutti i casi
        for ingrediente_redis in ingredienti_dati:
            if ingrediente == ingrediente_redis['raw_ingr']:
                nuovo_ingrediente = {
                    "raw_ingr": ingrediente_redis['raw_ingr'],
                    "foodItem": ingrediente_redis['foodItem'],
                    "Cals_per100grams": ingrediente_redis['Cals_per100grams'],
                    "quantity": random.randint(ingrediente_redis['quantity_min'], ingrediente_redis['quantity_max'])
                }
                nuovi_ingredienti.append(nuovo_ingrediente)
                break

    ricetta['ingredients'] = nuovi_ingredienti
    
with open('RAW_recipes_images_ingredients.json', 'w') as file:
    json.dump(ricette, file, indent=4)

Si elimina raw_ingr

In [15]:
with open('RAW_recipes_images_ingredients.json', 'r') as file:
    ricette = json.load(file)
    
for ricetta in ricette:
    for ingrediente in ricetta["ingredients"]:
        del ingrediente["raw_ingr"]
        
with open('RAW_recipes_images_ingredients_cleaned.json', 'w') as file:
    json.dump(ricette, file, indent=4)

In [16]:
with open('RAW_recipes_images_ingredients_cleaned.json', 'r') as file:
    ricette = json.load(file)
    
for ricetta in ricette:
    del ricetta["RecipeId"]
    del ricetta["AuthorName"]
        
with open('RAW_recipes_images_ingredients_cleaned1.json', 'w') as file:
    json.dump(ricette, file, indent=4)